<a href="https://colab.research.google.com/github/dhaev/Machine_Learning/blob/main/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KMeans and Agglomerative Clustering

This notebook demonstrates the use of KMeans and Agglomerative Clustering on the Ecoli dataset.


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from time import time

### **Dataset**
> This notebook uses an imbalanced dataset referred to as 'Ecoli' or 'Protein Localization Sites' dataset from the UCI repository [here](https://colab.research.google.com/github/dhaev/Machine_Learning/blob/main/clustering.ipynb#scrollTo=k7ujlGsP9TZy). We will first perform clustering with the highly under-represented classes and then perform clustering after removing the highly under-represented classes from the dataset.

1. **Title**: Ecoli (Protein Localization Sites).

2. **Creator and Maintainer**:
   - Kenta Nakai
     - Institute of Molecular and Cellular Biology
     - Osaka University
     - 1-3 Yamada-oka, Suita 565 Japan
     - [nakai@imcb.osaka-u.ac.jp](mailto:nakai@imcb.osaka-u.ac.jp)
     - [Website](http://www.imcb.osaka-u.ac.jp/nakai/psort.html)
   - Donor: Paul Horton ([paulh@cs.berkeley.edu](mailto:paulh@cs.berkeley.edu))
   - Date: September, 1996
   - See also: yeast database

3. **Past Usage**:
   - Reference: "A Probabilistic Classification System for Predicting the Cellular Localization Sites of Proteins", Paul Horton & Kenta Nakai, Intelligent Systems in Molecular Biology, 109-115. St. Louis, USA 1996.

4. **Number of Instances**: 336 for the E.coli dataset.

5. **Number of Attributes**: 8 (7 predictive, 1 name).

6. **Attribute Information**:
   - Sequence Name: Accession number for the SWISS-PROT database.
   - mcg: McGeoch's method for signal sequence recognition.
   - gvh: von Heijne's method for signal sequence recognition.
   - lip: von Heijne's Signal Peptidase II consensus sequence score.
   - chg: Presence of charge on N-terminus of predicted lipoproteins.
   - aac: Score of discriminant analysis of the amino acid content of outer membrane and periplasmic proteins.
   - alm1: Score of the ALOM membrane spanning region prediction program.
   - alm2: Score of ALOM program after excluding putative cleavable signal regions from the sequence.

7. **Missing Attribute Values**: None.

8. **Class Distribution**: The class is the localization site.
   - cp (cytoplasm) - 143
   - im (inner membrane without signal sequence) - 77
   - pp (perisplasm) - 52
   - imU (inner membrane, uncleavable signal sequence) - 35
   - om (outer membrane) - 20
   - omL (outer membrane lipoprotein) - 5
   - imL (inner membrane lipoprotein) - 2
   - imS (inner membrane, cleavable signal sequence) - 2

In [ ]:
# Define the file path and column names
filename = "/content/drive/MyDrive/machine_learning/ecoli/ecoli.data"
columns = ['Sequence Name', 'mcg', 'gvh', 'lip', 'chg', 'aac', 'alm1', 'alm2', 'class']

# Load the dataset into a pandas DataFrame
ecoli = pd.read_csv(filename, sep='\s+', names=columns)


### **Data Preprocessing 1**
* Check for missing values.
* Ensure all features have the correct data type.
* Remove the sequence name because it is unique in all instances and does not help in predicting or clustering.
* Split the features from the labels. Features are an (m x n) array where m is the number of instances and n is the number of features. Labels are flattened to a 1D array.
* Standardize the dataset.

In [ ]:
# Check for missing values
missing_values = ecoli.isnull().sum()
print("Missing values in each column")
print(30 * "_ ")
print(missing_values)

Missing values in each column
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
Sequence Name    0
mcg              0
gvh              0
lip              0
chg              0
aac              0
alm1             0
alm2             0
class            0
dtype: int64


In [ ]:
# Ensure all features have the correct data type
print("\nCheck data type")
print(30 * "_ ")
ecoli.info()


Check data type
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Sequence Name  336 non-null    object 
 1   mcg            336 non-null    float64
 2   gvh            336 non-null    float64
 3   lip            336 non-null    float64
 4   chg            336 non-null    float64
 5   aac            336 non-null    float64
 6   alm1           336 non-null    float64
 7   alm2           336 non-null    float64
 8   class          336 non-null    object 
dtypes: float64(7), object(2)
memory usage: 23.8+ KB


In [ ]:
# Drop the 'Sequence Name' column as it is unique for each instance and not useful for clustering
ecoli.drop(columns=['Sequence Name'], inplace=True)

# Split the features and labels
# Features are (m x n) array where m is the number of instances and n is the number of features
# Labels are flattened to a 1D array
data = ecoli.iloc[:, :-1]
labels = ecoli.iloc[:, -1].values.flatten()

# Print the shape of the data and the number of unique classes
(n_samples, n_features), n_classes = data.shape, np.unique(labels).size
print(f"# n_classes: {n_classes}; # samples: {n_samples}; # features {n_features}")

# n_classes: 8; # samples: 336; # features 7


## Benchmark Function

In [ ]:
def bench_clustering(cluster, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    cluster : cluster instance
        A :class:`~sklearn.cluster` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(), cluster).fit(data)
    fit_time = time() - t0
    if name.split(' ')[0] == 'Agglo':
      results = [name, fit_time]
    else:
      results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.fowlkes_mallows_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # Display results
    if name.split(' ')[0] == 'Agglo':
      formatter_result = (
          "{:9s}\t{:.3f}\t{:.3f}"
      )#\t{:.3f}
    else:
            formatter_result = (
          "{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}"
      )
    print(formatter_result.format(*results))



### **KMeans Clustering Implementation**
> There are two different KMeans clustering implementations. The parameters set are:

* `init`: Method for initialization (e.g., 'k-means++', 'random')
* `n_clusters`: Number of clusters to form
* `n_init`: Number of time the k-means algorithm will be run with different centroid seeds
* `random_state`: Determines random number generation for centroid initialization

The major difference between both implementations is the `init` method used. We then try different `n_init` parameters to observe how different `n_init` values affect the results in both implementations.

* The number of classes determines the number of clusters.

**Evaluation Metric**: Fowlkes-Mallows scores

In [ ]:
def kmeans_clustering(n_classes, data, labels, init_numbers):
    """Perform KMeans clustering with different initialization methods."""
    for n_init in init_numbers:
        print(f"n_init: {n_init}")
        print(120 * "_")
        print("init\t\ttime\tinertia\tFMI")

        # KMeans with k-means++ initialization
        kmeans = KMeans(init="k-means++", n_clusters=n_classes, n_init=n_init, random_state=0)
        bench_clustering(cluster=kmeans, name="k-means++", data=data, labels=labels)

        # KMeans with random initialization
        kmeans = KMeans(init="random", n_clusters=n_classes, n_init=n_init, random_state=0)
        bench_clustering(cluster=kmeans, name="random", data=data, labels=labels)

        print(120 * "_")



### **Agglomerative Clustering Implementation**
> There are four different Agglomerative Clustering implementations with three set parameters:

* `n_clusters`
* `metrics`
* `linkage`

All parameters except `linkage` remain the same in all implementations to observe how different linkage methods affect the results.

* The linkage parameters tested: 'ward', 'single', 'complete', 'average'.
* The Agglomerative metric used: 'euclidean'.
* The number of classes determines the number of clusters.

**Evaluation Metric**: Fowlkes-Mallows scores

In [ ]:
def agglomerative(n_classes, data, labels):
    """Perform Agglomerative Clustering with different linkage methods."""
    print(120 * "_")
    print("init\t\ttime\tFMI")

    # Agglomerative Clustering with ward linkage
    aggloward = AgglomerativeClustering(n_clusters=n_classes, metric='euclidean', linkage='ward')
    bench_clustering(cluster=aggloward, name="Agglo ward", data=data, labels=labels)

    # Agglomerative Clustering with single linkage
    agglosingle = AgglomerativeClustering(n_clusters=n_classes, metric='euclidean', linkage='single')
    bench_clustering(cluster=agglosingle, name="Agglo single", data=data, labels=labels)

    # Agglomerative Clustering with complete linkage
    agglocomplete = AgglomerativeClustering(n_clusters=n_classes, metric='euclidean', linkage='complete')
    bench_clustering(cluster=agglocomplete, name="Agglo complete", data=data, labels=labels)

    # Agglomerative Clustering with average linkage
    aggloaverage = AgglomerativeClustering(n_clusters=n_classes, metric='euclidean', linkage='average')
    bench_clustering(cluster=aggloaverage, name="Agglo average", data=data, labels=labels)

    print(120 * "_")



### **Clustering including highly under-represented classes**

### kmeans clustering

In [ ]:
# Perform KMeans clustering with different n_init values
kmeans_clustering(n_classes, data, labels, [1, 2, 3, 14])

n_init: 1
________________________________________________________________________________________________________________________
init		time	inertia	FMI
k-means++	0.261s	560	0.558
random   	0.074s	870	0.539
________________________________________________________________________________________________________________________
n_init: 2
________________________________________________________________________________________________________________________
init		time	inertia	FMI
k-means++	0.091s	560	0.558
random   	0.028s	796	0.534
________________________________________________________________________________________________________________________
n_init: 3
________________________________________________________________________________________________________________________
init		time	inertia	FMI
k-means++	0.036s	527	0.639
random   	0.018s	796	0.534
________________________________________________________________________________________________________________________
n_init: 14
__

n_init 1 had the best FMI score for k-mean++ which indicates agreement between clusters, whereas both n_init 1 and n_init 2 for random had the same values which were the best out of all n_init values for random. However kmean++ performed better

### agglomerative clustering

In [ ]:
agglomerative(n_classes,data,labels)

________________________________________________________________________________________________________________________
init		time	FMI
Agglo ward	0.046	0.635
Agglo single	0.021	0.529
Agglo complete	0.020	0.829
Agglo average	0.040	0.706
________________________________________________________________________________________________________________________


agglomerative clustering with complete linkage has the best FMI score

## **Clustering excluding highly under-represented classes**

### **Data Preprocessing 2**
> Remove under-represented classes

In [ ]:
#create a filter to remove  under-represented classes
filter = (ecoli['class']=='imL') | (ecoli['class']=='imS') | (ecoli['class']=='omL')
#apply filter to data
filtered_data = ecoli[~filter]

In [ ]:
#get feature data
data = filtered_data.iloc[:,:-1]
#get labels
labels = filtered_data.iloc[:,-1:].values.flatten()

(n_samples, n_features), n_classes = data.shape, np.unique(labels).size
print(f"# n_classes: {n_classes}; # samples: {n_samples}; # features {n_features}")

# n_classes: 5; # samples: 327; # features 7


### kmeans clustering

In [ ]:
# Perform KMeans clustering with different n_init values
kmeans_clustering(n_classes, data, labels, [1, 2, 3, 10, 12])

n_init: 1
________________________________________________________________________________________________________________________
init		time	inertia	FMI
k-means++	0.027s	646	0.780
random   	0.021s	847	0.806
________________________________________________________________________________________________________________________
n_init: 2
________________________________________________________________________________________________________________________
init		time	inertia	FMI
k-means++	0.038s	646	0.780
random   	0.016s	847	0.806
________________________________________________________________________________________________________________________
n_init: 3
________________________________________________________________________________________________________________________
init		time	inertia	FMI
k-means++	0.032s	646	0.779
random   	0.032s	847	0.806
________________________________________________________________________________________________________________________
n_init: 10
__

### agglomerative clustering

In [ ]:
agglomerative(n_classes,data,labels)

________________________________________________________________________________________________________________________
init		time	FMI
Agglo ward	0.024	0.819
Agglo single	0.016	0.532
Agglo complete	0.023	0.762
Agglo average	0.026	0.814
________________________________________________________________________________________________________________________


agglomerative clustering with ward linkage had the best FMI score

## Clustering Comparisons Summary

**KMeans Clustering**:
- **Initialization Methods**: Two different initialization methods were tested: 'k-means++' and 'random'.
- **n_init Values**: Various `n_init` values were tested to observe their impact on the results.
- **Evaluation Metric**: Fowlkes-Mallows Index (FMI) scores were used to evaluate the clustering performance.
- **Results**:
  - The 'k-means++' initialization generally performed better than the 'random' initialization.
  - The best FMI score for 'k-means++' was achieved with `n_init = 1`, indicating good agreement between clusters.
  - For 'random' initialization, the best FMI scores were observed with `n_init = 1` and `n_init = 2`.

**Agglomerative Clustering**:
- **Linkage Methods**: Four different linkage methods were tested: 'ward', 'single', 'complete', and 'average'.
- **Metric**: The 'euclidean' metric was used for all linkage methods.
- **Evaluation Metric**: Fowlkes-Mallows Index (FMI) scores were used to evaluate the clustering performance.
- **Results**:
  - The 'complete' linkage method achieved the highest FMI score when including highly under-represented classes.
  - The 'ward' linkage method achieved the highest FMI score when excluding highly under-represented classes.

**Overall**:
- **Agglomerative Clustering**: Performed better than both KMeans implementations based on FMI scores, both when including and excluding under-represented classes.
